In [2]:
import numpy as np

In [7]:
ndim = 2
rhs = np.arange(30).reshape((6, 5))
igs = [2, 2]
factor = 100
wall_idx = np.empty((ndim), dtype=object)
for dim in range(ndim):
    wall_idx[dim] = slice(igs[dim], -igs[dim])
print(rhs)
for dim in range(ndim):
    if True:
        for direction in [-1, 1]:
            wall_idx[dim] = (igs[dim] - 1) * direction
            if direction == -1:
                wall_idx[dim] -= 1
            wall_idx_tuple = tuple(wall_idx)
            rhs[wall_idx_tuple] *= factor
            print(wall_idx_tuple)

print(rhs)

[[ 0  1  2  3  4]
 [ 5  6  7  8  9]
 [10 11 12 13 14]
 [15 16 17 18 19]
 [20 21 22 23 24]
 [25 26 27 28 29]]
(-2, slice(2, -2, None))
(1, slice(2, -2, None))
(1, -2)
(1, 1)
[[   0    1    2    3    4]
 [   5  600  700  800    9]
 [  10   11   12   13   14]
 [  15   16   17   18   19]
 [  20   21 2200   23   24]
 [  25   26   27   28   29]]


In [41]:
def euler_backward_non_advective_impl_part(Sol, mpv, elem, node, ud, th, t, dt, alpha_diff, Sol0 = None, writer = None, label=None, debug=False):
    nc = node.sc
    rhs = mpv.rhs

    if Sol0 is not None:
        bdry.set_explicit_boundary_data(Sol0, elem, ud, th, mpv)
        operator_coefficients_nodes(elem, node, Sol0, mpv, ud, th, dt)
    else:
        bdry.set_explicit_boundary_data(Sol, elem, ud, th, mpv)
        operator_coefficients_nodes(elem, node, Sol, mpv, ud, th, dt)

    bdry.set_ghostnodes_p2(mpv.p2_nodes,node,ud)
    correction_nodes(Sol,elem,node,mpv,mpv.p2_nodes,dt,ud,th,0)
    bdry.set_explicit_boundary_data(Sol, elem, ud, th, mpv)

    rhs[...] = divergence_nodes(rhs,elem,node,Sol,ud)

    rhs /= dt

    if ud.is_compressible == 0:
        if ud.is_ArakawaKonor:
            rhs -= mpv.wcenter * mpv.dp2_nodes
            mpv.wcenter[...] = 0.0
        else:
            mpv.wcenter[...] *= ud.compressibility
    else:
        mpv.wcenter *= ud.compressibility

    mpv.rhs[...] = rhs

    VS = True

    # prepare initial left-hand side and the laplacian stencil
    if elem.ndim == 2:
        Vec = mpv
        coriolis_params = multiply_inverse_coriolis(Vec, Sol, mpv, ud, elem, node, dt, attrs=('u', 'v', 'w'), get_coeffs = True)
        # lap = stencil_9pt(elem,node,mpv,Sol,ud,diag_inv,dt,coriolis_params)
        # sh = (ud.inx)*(ud.iny)

        diag_inv = lm_lp.precon_diag_prepare(mpv, elem, node, ud, coriolis_params)
        rhs *= diag_inv

        p2 = mpv.p2_nodes[node.i2].T
        lap = lm_lp.stencil_9pt_numba_test(mpv,node,coriolis_params,diag_inv, ud)
        sh = p2.shape[0] * p2.shape[1]

    elif elem.ndim == 3 and elem.icy - 2*elem.igs[1] <= 2:
        # horizontal slice hack
        p2 = np.copy(mpv.p2_nodes[1:-1,elem.igs[1],1:-1])
        lap = lm_lp.stencil_hs(elem,node,mpv,ud,diag_inv,dt)
        sh = p2.reshape(-1).shape[0]

    elif elem.ndim == 3 and elem.iicy > 1 and elem.iicz == 1:
        # vertical slice hack

        if not VS:
            lap = lm_lp.stencil_vs(elem,node,mpv,ud,diag_inv,dt)
            sh = (node.iicx)*(node.iicy)
        if VS:
            p2 = np.copy(mpv.p2_nodes[1:-1,1:-1,elem.igs[2]])
            lap = lm_lp.stencil_vs(elem,node,mpv,ud,diag_inv,dt)
            sh = p2.reshape(-1).shape[0]

    elif elem.ndim == 3 and elem.icy - 2*elem.igs[1] > 2:
        lap = lm_lp.stencil_27pt(elem,node,mpv,ud,diag_inv,dt)
        sh = p2.reshape(-1).shape[0]

    lap = sp.sparse.linalg.LinearOperator((sh,sh),lap)
    # lap = LinearOperator(sh,lap)

    counter = solver_counter()

    # prepare right-hand side
    if elem.ndim == 2:
        rhs_inner = rhs[1:-1,1:-1].T.ravel()
    elif elem.ndim == 3 and elem.iicy > 1 and elem.iicz == 1:

        if not VS:
            rhs_inner = rhs[...,elem.igs[2]][node.igx:-node.igx,node.igy:-node.igy].ravel()
        if VS:
            rhs_inner = rhs[1:-1,1:-1,elem.igs[2]].ravel()

    elif elem.ndim == 3 and elem.icy - 2*elem.igs[1] > 2:
        rhs_inner = rhs[1:-1,1:-1,1:-1].ravel()
    else:
        rhs_inner = rhs[1:-1,elem.igs[1],1:-1].ravel()

    p2, _ = sp.sparse.linalg.bicgstab(lap,rhs_inner,tol=ud.tol,maxiter=ud.max_iterations,callback=counter)

    global total_calls, total_iter
    total_iter += counter.niter
    total_calls += 1
    p2_full = np.zeros(nc).squeeze()
    if elem.ndim == 2:
        p2_full[node.i2] = p2.reshape(rhs[node.i1].shape[1],rhs[node.i1].shape[0]).T
    elif elem.ndim == 3 and elem.icy - 2*elem.igs[1] <= 2:
        # horizontal slice hack
        p2 = p2.reshape(ud.inx+2, ud.inz+2)
        p2 = np.expand_dims(p2,1)
        p2 = np.repeat(p2, node.icy, axis=1)
        p2_full[1:-1,:,1:-1] = p2
    elif elem.ndim == 3 and elem.iicy > 1 and elem.iicz == 1:
        if not VS:
            p2 = p2.reshape(node.iicx,node.iicy)
            p2 = np.repeat(p2[...,np.newaxis], node.icz, axis=2)
            p2_full[node.igx:-node.igx,node.igy:-node.igy] = p2
        if VS:
            p2 = p2.reshape(ud.inx+2, ud.iny+2)
            p2 = np.expand_dims(p2,2)
            p2 = np.repeat(p2, node.icz, axis=2)
            p2_full[1:-1,1:-1,:] = p2

    elif elem.ndim == 3 and elem.icy - 2*elem.igs[1] > 2:
        p2_full[1:-1,1:-1,1:-1] = p2.reshape(ud.inx+2,ud.iny+2,ud.inz+2)

    bdry.set_ghostnodes_p2(p2_full,node,ud)
    correction_nodes(Sol,elem,node,mpv,p2_full,dt,ud,th,1)

    mpv.p2_nodes[...] += p2_full
    bdry.set_ghostnodes_p2(mpv.p2_nodes,node,ud)
    bdry.set_explicit_boundary_data(Sol, elem, ud, th, mpv)


In [ ]:
import numba as nb

In [16]:
def stencil_9pt(elem,node,mpv,Sol,ud,diag_inv,dt,coriolis_params):
    igx = elem.igx
    igy = elem.igy

    icxn = node.icx
    icyn = node.icy

    iicxn = icxn - (2 * igx)
    iicyn = icyn - (2 * igy)

    iicxn, iicyn = iicyn, iicxn

    dx = node.dy
    dy = node.dx

    inner_domain = (slice(igx,-igx),slice(igy,-igy))
    i1 = node.i1

    hplusx = mpv.wplus[1][i1].reshape(-1,)
    hplusy = mpv.wplus[0][i1].reshape(-1,)
    hcenter = mpv.wcenter[i1].reshape(-1,)

    diag_inv = diag_inv[i1].reshape(-1,)

    oodx = 1.0 / (dx)
    oody = 1.0 / (dy)

    x_periodic = ud.bdry_type[1] == opts.BdryType.PERIODIC
    y_periodic = ud.bdry_type[0] == opts.BdryType.PERIODIC

    x_wall = ud.bdry_type[1] == opts.BdryType.WALL or ud.bdry_type[1] == opts.BdryType.RAYLEIGH
    y_wall = ud.bdry_type[0] == opts.BdryType.WALL or ud.bdry_type[0] == opts.BdryType.RAYLEIGH
    return lambda p : lap2D_gather(p, igx,igy, iicxn, iicyn, hplusx, hplusy, hcenter, oodx, oody, x_periodic, y_periodic, x_wall, y_wall, diag_inv, coriolis_params)

@nb.jit(nopython=True, nogil=False, cache=True)
def lap2D_gather(p, igx,igy, iicxn, iicyn, hplusx, hplusy, hcenter, oodx, oody, x_periodic, y_periodic, x_wall, y_wall, diag_inv, coriolis):
    ngnc = (iicxn) * (iicyn)
    lap = np.zeros((ngnc))
    cnt_x = 0
    cnt_y = 0

    nine_pt = 0.25 * (2.0) * 1.0
    cyy, cxx, cyx, cxy = coriolis
    oodx2 = 0.5 * oodx**2
    oody2 = 0.5 * oody**2

    for idx in range(iicxn * iicyn):
        ne_topleft = idx - iicxn - 1
        ne_topright = idx - iicxn
        ne_botleft = idx - 1
        ne_botright = idx

        # get indices of the 9pt stencil
        topleft_idx = idx - iicxn - 1
        midleft_idx = idx - 1
        botleft_idx = idx + iicxn - 1

        topmid_idx = idx - iicxn
        midmid_idx = idx
        botmid_idx = idx + iicxn

        topright_idx = idx - iicxn + 1
        midright_idx = idx + 1
        botright_idx = idx + iicxn + 1

        if cnt_x == 0:
            topleft_idx += iicxn - 1
            midleft_idx += iicxn - 1
            botleft_idx += iicxn - 1

            ne_topleft += iicxn - 1
            ne_botleft += iicxn - 1

        if cnt_x == (iicxn - 1):
            topright_idx -= iicxn - 1
            midright_idx -= iicxn - 1
            botright_idx -= iicxn - 1

            ne_topright -= iicxn - 1
            ne_botright -= iicxn - 1

        if cnt_y == 0:
            topleft_idx += ((iicxn) * (iicyn - 1))
            topmid_idx += ((iicxn) * (iicyn - 1))
            topright_idx += ((iicxn) * (iicyn - 1))

            ne_topleft += ((iicxn) * (iicyn - 1))
            ne_topright += ((iicxn) * (iicyn - 1))

        if cnt_y == (iicyn - 1):
            botleft_idx -= ((iicxn) * (iicyn - 1))
            botmid_idx -= ((iicxn) * (iicyn - 1))
            botright_idx -= ((iicxn) * (iicyn - 1))

            ne_botleft -= ((iicxn) * (iicyn - 1))
            ne_botright -= ((iicxn) * (iicyn - 1))

        topleft = p[topleft_idx]
        midleft = p[midleft_idx]
        botleft = p[botleft_idx]

        topmid = p[topmid_idx]
        midmid = p[midmid_idx]
        botmid = p[botmid_idx]

        topright = p[topright_idx]
        midright = p[midright_idx]
        botright = p[botright_idx]

        hplusx_topleft = hplusx[ne_topleft]
        hplusx_botleft = hplusx[ne_botleft]
        hplusy_topleft = hplusy[ne_topleft]
        hplusy_botleft = hplusy[ne_botleft]

        hplusx_topright = hplusx[ne_topright]
        hplusx_botright = hplusx[ne_botright]
        hplusy_topright = hplusy[ne_topright]
        hplusy_botright = hplusy[ne_botright]

        cxx_tl  = cxx[ne_topleft]
        cxx_tr = cxx[ne_topright]
        cxx_bl  = cxx[ne_botleft]
        cxx_br = cxx[ne_botright]

        cxy_tl  = cxy[ne_topleft]
        cxy_tr  = cxy[ne_topright]
        cxy_bl  = cxy[ne_botleft]
        cxy_br  = cxy[ne_botright]

        cyx_tl  = cyx[ne_topleft]
        cyx_tr  = cyx[ne_topright]
        cyx_bl  = cyx[ne_botleft]
        cyx_br  = cyx[ne_botright]

        cyy_tl  = cyy[ne_topleft]
        cyy_tr  = cyy[ne_topright]
        cyy_bl  = cyy[ne_botleft]
        cyy_br  = cyy[ne_botright]

        Dx_tl = 0.5 * (topmid   - topleft + midmid   - midleft) * hplusx_topleft
        Dx_tr = 0.5 * (topright - topmid  + midright - midmid ) * hplusx_topright
        Dx_bl = 0.5 * (botmid   - botleft + midmid   - midleft) * hplusx_botleft
        Dx_br = 0.5 * (botright - botmid  + midright - midmid ) * hplusx_botright

        Dy_tl = 0.5 * (midmid   - topmid   + midleft - topleft) * hplusy_topleft
        Dy_tr = 0.5 * (midright - topright + midmid  - topmid ) * hplusy_topright
        Dy_bl = 0.5 * (botmid   - midmid   + botleft - midleft) * hplusy_botleft
        Dy_br = 0.5 * (botright - midright + botmid  - midmid ) * hplusy_botright

        fac = 1.0
        Dxx = 0.5 * (cxx_tr * Dx_tr - cxx_tl * Dx_tl + cxx_br * Dx_br - cxx_bl * Dx_bl) * oodx * oodx * fac
        Dyy = 0.5 * (cyy_br * Dy_br - cyy_tr * Dy_tr + cyy_bl * Dy_bl - cyy_tl * Dy_tl) * oody * oody * fac
        Dyx = 0.5 * (cyx_br * Dy_br - cyx_bl * Dy_bl + cyx_tr * Dy_tr - cyx_tl * Dy_tl) * oody * oodx * fac
        Dxy = 0.5 * (cxy_br * Dx_br - cxy_tr * Dx_tr + cxy_bl * Dy_bl - cxy_tl * Dx_tl) * oodx * oody * fac


        lap[idx] = Dxx + Dyy + Dyx + Dxy + hcenter[idx] * p[idx]


Norm of difference: 7.038508264717789


In [26]:
import numpy as np
import numba as nb

# Define constants for clarity
HALF = 0.5

@nb.jit(nopython=True, nogil=True, cache=True, fastmath=False) # nogil=True should be safe
def lap2D_gather_refactored_numba(
    p_node: np.ndarray,      # Input pressure vector (flattened internal nodes)
    iicxn: int,              # Number of internal nodes in x
    iicyn: int,              # Number of internal nodes in y
    hplusx_elem: np.ndarray, # Coefficient array (ELEMENT-centered or face-centered)
    hplusy_elem: np.ndarray, # Coefficient array (ELEMENT-centered or face-centered)
    hcenter_node: np.ndarray,# Coefficient array (NODE-centered)
    oodx: float,             # 1 / dx
    oody: float,             # 1 / dy
    x_periodic: bool,        # Boolean or int flag
    y_periodic: bool,        # Boolean or int flag
    x_wall: bool,            # Boolean or int flag
    y_wall: bool,            # Boolean or int flag
    diag_inv_node: np.ndarray,# Diagonal preconditioner (NODE-centered)
    coriolis_elem: tuple,    # Tuple of ELEMENT-centered coeff arrays (cyy, cxx, cyx, cxy)
    out: np.ndarray = None   # Optional output array for in-place operation
) -> np.ndarray:
    """
    Calculates the action of a 2D discretized operator (Laplacian + Coriolis)
    on a vector 'p_node' using a 9-point stencil (matrix-free via Numba).

    Handles periodic and wall boundary conditions by adjusting coefficients.
    Operates on flattened arrays corresponding to internal grid nodes/elements.

    Args:
        p_node: Flattened 1D array of input values at internal nodes.
        iicxn, iicyn: Internal grid dimensions (number of nodes).
        hplusx_elem, hplusy_elem: Coefficient arrays, assumed mapped to elements
                                 (e.g., indexed by bottom-left node's internal index).
                                 Size should match number of internal elements.
        hcenter_node: Central coefficient at each internal node. Size ngnc.
        oodx, oody: Grid spacing reciprocals.
        x_periodic, y_periodic: Flags for periodic boundaries.
        x_wall, y_wall: Flags for wall boundaries (assumed zero Neumann).
        diag_inv_node: Diagonal preconditioning values at each internal node. Size ngnc.
        coriolis_elem: Tuple (cyy, cxx, cyx, cxy) of coefficient arrays,
                       assumed mapped to elements.
        out: Optional pre-allocated array (size ngnc) to store the result.

    Returns:
        A 1D numpy array (size ngnc) containing the result of the operator
        applied to p_node. If 'out' was provided, it returns 'out'.
    """
    ngnc = iicxn * iicyn # Total number of internal nodes

    # Initialize output array
    if out is None:
        lap_node = np.zeros(ngnc, dtype=p_node.dtype)
    else:
        # Ensure the output array has the correct shape if provided
        if out.shape[0] != ngnc:
            # Numba doesn't support raising exceptions easily, maybe return zeros or handle outside
            return np.zeros(ngnc, dtype=p_node.dtype) # Indicate error state
        lap_node = out
        lap_node[:] = 0.0 # Zero out array if accumulating isn't desired

    # Unpack Coriolis terms for clarity
    cyy_elem, cxx_elem, cyx_elem, cxy_elem = coriolis_elem

    # Use nested loops for better 2D clarity
    for j in range(iicyn):  # Loop over y index (internal nodes, 0 to iicyn-1)
        for i in range(iicxn):  # Loop over x index (internal nodes, 0 to iicxn-1)
            # --- Calculate current node index ---
            idx_mid = j * iicxn + i

            # --- Determine neighbor node coordinates (ni, nj) with periodic wrapping ---
            # Note: Python's % handles negative numbers correctly for wrapping in Numba too
            i_w = (i - 1 + iicxn) % iicxn if x_periodic else i - 1
            i_e = (i + 1) % iicxn         if x_periodic else i + 1
            j_s = (j - 1 + iicyn) % iicyn if y_periodic else j - 1
            j_n = (j + 1) % iicyn         if y_periodic else j + 1

            # --- Calculate flattened neighbor indices in 'p_node' array ---
            # These indices are potentially invalid if walls are present,
            # but we fetch p values first and apply BCs via coefficients later.
            idx_mm = idx_mid           # Middle-Middle (center)
            idx_wm = j * iicxn + i_w   # West-Middle
            idx_em = j * iicxn + i_e   # East-Middle
            idx_ms = j_s * iicxn + i   # Middle-South
            idx_mn = j_n * iicxn + i   # Middle-North
            idx_ws = j_s * iicxn + i_w # West-South
            idx_es = j_s * iicxn + i_e # East-South
            idx_wn = j_n * iicxn + i_w # West-North
            idx_en = j_n * iicxn + i_e # East-North

            # --- Boundary Checks for Non-Periodic Walls ---
            # These flags determine if a neighbor is *logically* outside a wall
            on_west_boundary  = (not x_periodic and i == 0)
            on_east_boundary  = (not x_periodic and i == iicxn - 1)
            on_south_boundary = (not y_periodic and j == 0)
            on_north_boundary = (not y_periodic and j == iicyn - 1)

            # --- Gather 'p' values from neighbors ---
            # For wall boundaries, fetch the value from the boundary node itself (p[idx_mid])
            # This effectively applies the Neumann BC when combined with zeroed coefficients later.
            p_mm = p_node[idx_mm]
            p_wm = p_node[idx_wm if not on_west_boundary else idx_mm]
            p_em = p_node[idx_em if not on_east_boundary else idx_mm]
            p_ms = p_node[idx_ms if not on_south_boundary else idx_mm]
            p_mn = p_node[idx_mn if not on_north_boundary else idx_mm]

            p_ws = p_node[idx_ws if not (on_west_boundary or on_south_boundary) else idx_mm]
            p_es = p_node[idx_es if not (on_east_boundary or on_south_boundary) else idx_mm]
            p_wn = p_node[idx_wn if not (on_west_boundary or on_north_boundary) else idx_mm]
            p_en = p_node[idx_en if not (on_east_boundary or on_north_boundary) else idx_mm]


            # --- Determine element indices for coefficients ---
            # Assuming coeffs arrays (hplus, coriolis) are mapped based on the
            # bottom-left node's *internal* index.
            # Calculate the potential indices of the 4 surrounding elements' bottom-left nodes.
            # Need to handle periodicity/walls for these indices if coeff arrays only store internal elements.
            elem_idx_sw = ((j - 1 + iicyn) % iicyn if y_periodic else j - 1) * iicxn + \
                          ((i - 1 + iicxn) % iicxn if x_periodic else i - 1)
            elem_idx_se = ((j - 1 + iicyn) % iicyn if y_periodic else j - 1) * iicxn + \
                          (i % iicxn if x_periodic else i) # Use i directly for SE/NE x-index
            elem_idx_nw = (j % iicyn if y_periodic else j) * iicxn + \
                          ((i - 1 + iicxn) % iicxn if x_periodic else i - 1)
            elem_idx_ne = (j % iicyn if y_periodic else j) * iicxn + \
                          (i % iicxn if x_periodic else i) # Current node (j,i) is bottom-left of NE element

            # --- Gather Coefficients (hplus, coriolis) ---
            # Use helper function to safely get coefficients, handling potential
            # out-of-bounds access for wall boundaries by returning 0.0

            # SW Element Coefficients
            valid_sw = (not (on_west_boundary or on_south_boundary))
            hpx_sw = hplusx_elem[elem_idx_sw] if valid_sw else 0.0
            hpy_sw = hplusy_elem[elem_idx_sw] if valid_sw else 0.0
            cxx_sw = cxx_elem[elem_idx_sw] if valid_sw else 0.0
            cyy_sw = cyy_elem[elem_idx_sw] if valid_sw else 0.0
            cxy_sw = cxy_elem[elem_idx_sw] if valid_sw else 0.0
            cyx_sw = cyx_elem[elem_idx_sw] if valid_sw else 0.0

            # SE Element Coefficients
            valid_se = (not (on_east_boundary or on_south_boundary))
            hpx_se = hplusx_elem[elem_idx_se] if valid_se else 0.0
            hpy_se = hplusy_elem[elem_idx_se] if valid_se else 0.0
            cxx_se = cxx_elem[elem_idx_se] if valid_se else 0.0
            cyy_se = cyy_elem[elem_idx_se] if valid_se else 0.0
            cxy_se = cxy_elem[elem_idx_se] if valid_se else 0.0
            cyx_se = cyx_elem[elem_idx_se] if valid_se else 0.0

            # NW Element Coefficients
            valid_nw = (not (on_west_boundary or on_north_boundary))
            hpx_nw = hplusx_elem[elem_idx_nw] if valid_nw else 0.0
            hpy_nw = hplusy_elem[elem_idx_nw] if valid_nw else 0.0
            cxx_nw = cxx_elem[elem_idx_nw] if valid_nw else 0.0
            cyy_nw = cyy_elem[elem_idx_nw] if valid_nw else 0.0
            cxy_nw = cxy_elem[elem_idx_nw] if valid_nw else 0.0
            cyx_nw = cyx_elem[elem_idx_nw] if valid_nw else 0.0

            # NE Element Coefficients
            valid_ne = (not (on_east_boundary or on_north_boundary))
            hpx_ne = hplusx_elem[elem_idx_ne] if valid_ne else 0.0
            hpy_ne = hplusy_elem[elem_idx_ne] if valid_ne else 0.0
            cxx_ne = cxx_elem[elem_idx_ne] if valid_ne else 0.0
            cyy_ne = cyy_elem[elem_idx_ne] if valid_ne else 0.0
            cxy_ne = cxy_elem[elem_idx_ne] if valid_ne else 0.0
            cyx_ne = cyx_elem[elem_idx_ne] if valid_ne else 0.0

            # --- Calculate fluxes / derivatives (using gathered p and coeffs) ---
            # These coefficients (hpx_*, cxx_*, etc.) are now correctly zeroed
            # if they correspond to an element outside a wall boundary.
            Dx_tl = HALF * (p_mn - p_wn + p_mm - p_wm) * hpx_nw # Top-Left face flux approx
            Dx_tr = HALF * (p_en - p_mn + p_em - p_mm) * hpx_ne # Top-Right face flux approx
            Dx_bl = HALF * (p_mm - p_wm + p_ms - p_ws) * hpx_sw # Bottom-Left face flux approx
            Dx_br = HALF * (p_em - p_mm + p_es - p_ms) * hpx_se # Bottom-Right face flux approx

            Dy_tl = HALF * (p_mm - p_mn + p_wm - p_wn) * hpy_nw # Top-Left face flux approx
            Dy_tr = HALF * (p_em - p_en + p_mm - p_mn) * hpy_ne # Top-Right face flux approx
            Dy_bl = HALF * (p_ms - p_mm + p_ws - p_wm) * hpy_sw # Bottom-Left face flux approx
            Dy_br = HALF * (p_es - p_em + p_ms - p_mm) * hpy_se # Bottom-Right face flux approx

            # --- Combine terms for the operator action at 'idx_mid' ---
            # Based on divergence form: d/dx(F_x) + d/dy(F_y)
            # F_x approx = cxx * Dx + cxy * Dy
            # F_y approx = cyx * Dx + cyy * Dy
            # d/dx(F_x) approx = ( F_x_east_face - F_x_west_face ) * oodx
            # d/dy(F_y) approx = ( F_y_north_face - F_y_south_face ) * oody

            # Approximate fluxes at cell faces around node (i,j)
            Fx_west = HALF * (cxx_sw * Dx_bl + cxx_nw * Dx_tl + cxy_sw * Dy_bl + cxy_nw * Dy_tl)
            Fx_east = HALF * (cxx_se * Dx_br + cxx_ne * Dx_tr + cxy_se * Dy_br + cxy_ne * Dy_tr)
            Fy_south = HALF * (cyx_sw * Dx_bl + cyx_se * Dx_br + cyy_sw * Dy_bl + cyy_se * Dy_br)
            Fy_north = HALF * (cyx_nw * Dx_tl + cyx_ne * Dx_tr + cyy_nw * Dy_tl + cyy_ne * Dy_tr)

            divergence = (Fx_east - Fx_west) * oodx + (Fy_north - Fy_south) * oody

            # --- Final value for the node ---
            # Check if the original formula includes the hcenter term *before* or *after* divergence.
            # Assuming it's added:
            lap_val = divergence + hcenter_node[idx_mid] * p_mm

            # Apply diagonal preconditioner
            lap_node[idx_mid] = lap_val * diag_inv_node[idx_mid]

    return lap_node

In [28]:
import numpy as np
import time
import sys
#
# # --- Assume functions are defined/imported ---
# # Make sure these placeholder functions have the EXACT same signature
# # as your actual functions. Replace the imports/definitions below.
#
# # Placeholder for the ORIGINAL Numba function
# try:
#     # Option 1: If it's in a file named, e.g., original_laplacian.py
#     from original_laplacian import lap2D_gather as lap2D_gather_original
#     print("Imported lap2D_gather_original from original_laplacian.py")
# except ImportError:
#     print("Warning: Could not import lap2D_gather_original. Using placeholder.")
#     # Option 2: Define a dummy placeholder if the real one isn't available
#     # This will likely fail comparisons but allows the script structure to run.
#     import numba as nb
#     @nb.jit(nopython=True)
#     def lap2D_gather_original(p, igx,igy, iicxn, iicyn, hplusx, hplusy, hcenter, oodx, oody, x_periodic, y_periodic, x_wall, y_wall, diag_inv, coriolis):
#         # This placeholder just returns zeros - REPLACE IT with your original code
#         # Note: The signature must match EXACTLY, including parameter names if
#         # the original code relied on them implicitly (though less likely in Numba).
#         # The dummy 'igx', 'igy' are included here assuming they were in the original
#         # even if unused, to maintain the signature for the call.
#         ngnc = iicxn * iicyn
#         return np.zeros(ngnc, dtype=p.dtype)
#
# # Placeholder for the NEW Refactored Numba function
# try:
#     # Option 1: If it's in a file named, e.g., refactored_laplacian.py
#     from refactored_laplacian import lap2D_gather_refactored_numba
#     print("Imported lap2D_gather_refactored_numba from refactored_laplacian.py")
# except ImportError:
#     print("Warning: Could not import lap2D_gather_refactored_numba. Using placeholder.")
#     # Option 2: Define a dummy placeholder
#     import numba as nb
#     @nb.jit(nopython=True)
#     def lap2D_gather_refactored_numba(p_node: np.ndarray, iicxn: int, iicyn: int, hplusx_elem: np.ndarray, hplusy_elem: np.ndarray, hcenter_node: np.ndarray, oodx: float, oody: float, x_periodic: bool, y_periodic: bool, x_wall: bool, y_wall: bool, diag_inv_node: np.ndarray, coriolis_elem: tuple, out: np.ndarray = None):
#         # This placeholder just returns zeros - REPLACE IT with your refactored code
#         ngnc = iicxn * iicyn
#         if out is None:
#             out = np.zeros(ngnc, dtype=p_node.dtype)
#         else:
#             out[:] = 0.0
#         return out
# # --- End Placeholder functions ---


def run_numba_comparison(iicxn, iicyn, num_trials=10):
    """
    Compares the original and refactored Numba implementations.
    """
    print("-" * 60)
    print(f"Running comparison for grid size: {iicxn} x {iicyn}")
    print(f"Number of trials for timing: {num_trials}")
    print("-" * 60)

    ngnc = iicxn * iicyn # Number of internal grid nodes
    nelem = iicxn * iicyn # Assuming element count matches internal nodes
    # Dummy ghost cell info if needed by original signature (adjust if different)
    igx, igy = 2, 2

    # --- Generate Consistent Random Test Data (CPU) ---
    np.random.seed(42)
    p_np = np.random.rand(ngnc).astype(np.float64)
    hplusx_np = np.random.rand(nelem).astype(np.float64) + 0.1
    hplusy_np = np.random.rand(nelem).astype(np.float64) + 0.1
    hcenter_np = np.random.rand(ngnc).astype(np.float64) * 5.0 + 1.0
    diag_inv_np = (1.0 / hcenter_np).astype(np.float64)
    coriolis_np = tuple([ (np.random.rand(nelem).astype(np.float64) - 0.5) * 0.1 for _ in range(4) ])

    oodx = float(iicxn)
    oody = float(iicyn)
    x_periodic, y_periodic = True, True
    x_wall, y_wall = False, False

    # --- 1. Run Original Numba Version ---
    print("Running Original Numba version...")
    try:
        # Warm-up call
        _ = lap2D_gather(p_np, igx, igy, iicxn, iicyn, hplusx_np, hplusy_np, hcenter_np, oodx, oody,
                         x_periodic, y_periodic, x_wall, y_wall, diag_inv_np, coriolis_np)

        start_time_orig = time.perf_counter()
        for _ in range(num_trials):
            result_orig = lap2D_gather(p_np, igx, igy, iicxn, iicyn, hplusx_np, hplusy_np, hcenter_np, oodx, oody,
                                     x_periodic, y_periodic, x_wall, y_wall, diag_inv_np, coriolis_np)
        end_time_orig = time.perf_counter()
        time_orig_avg = (end_time_orig - start_time_orig) / num_trials
        print(f"Original average execution time: {time_orig_avg:.6f} seconds")
    except Exception as e:
        print(f"Error running original version: {e}")
        time_orig_avg = float('inf')
        result_orig = None


    # --- 2. Run Refactored Numba Version ---
    print("\nRunning Refactored Numba version...")
    try:
        # Optional output buffer for refactored version
        out_buffer = np.zeros_like(p_np)
        # Warm-up call
        _ = lap2D_gather_refactored_numba(p_np, iicxn, iicyn, hplusx_np, hplusy_np, hcenter_np, oodx, oody,
                                x_periodic, y_periodic, x_wall, y_wall, diag_inv_np, coriolis_np, out=out_buffer)

        start_time_ref = time.perf_counter()
        for _ in range(num_trials):
            # Example using the output buffer
            result_ref = lap2D_gather_refactored_numba(p_np, iicxn, iicyn, hplusx_np, hplusy_np, hcenter_np, oodx, oody,
                                           x_periodic, y_periodic, x_wall, y_wall, diag_inv_np, coriolis_np, out=out_buffer)
            # Or call without 'out' if preferred:
            # result_ref = lap2D_gather_refactored_numba(p_np, iicxn, iicyn, hplusx_np, hplusy_np, hcenter_np, oodx, oody,
            #                                x_periodic, y_periodic, x_wall, y_wall, diag_inv_np, coriolis_np)
        end_time_ref = time.perf_counter()
        time_ref_avg = (end_time_ref - start_time_ref) / num_trials
        print(f"Refactored average execution time: {time_ref_avg:.6f} seconds")
        # If using 'out', result_ref is the same buffer 'out_buffer'
        if 'out_buffer' in locals() and result_ref is out_buffer:
             result_ref = out_buffer.copy() # Copy if needed for comparison, as buffer is overwritten

    except Exception as e:
        print(f"Error running refactored version: {e}")
        time_ref_avg = float('inf')
        result_ref = None

    # --- 3. Compare Results ---
    print("\nComparing results...")
    if result_orig is not None and result_ref is not None:
        try:
            are_close = np.allclose(result_orig, result_ref, rtol=1e-9, atol=1e-12) # Use tighter tolerance
            print(f"Results are close: {are_close}")
            if not are_close:
                diff = np.abs(result_orig - result_ref)
                max_abs_diff = np.max(diff)
                # Calculate relative difference carefully
                # Add small epsilon to denominator to avoid division by zero/large rel diff for small numbers
                denom = np.maximum(np.abs(result_orig), np.abs(result_ref)) + 1e-15
                max_rel_diff = np.max(diff / denom)

                print(f"  Max absolute difference: {max_abs_diff:.2e}")
                print(f"  Max relative difference: {max_rel_diff:.2e}")
                # Optionally print indices where differences occur
                # diff_indices = np.where(~np.isclose(result_orig, result_ref, rtol=1e-9, atol=1e-12))[0]
                # print(f"  Indices with differences (first few): {diff_indices[:10]}")
        except Exception as e:
            print(f"Error during comparison: {e}")
            print(f"  Original result shape: {result_orig.shape}, dtype: {result_orig.dtype}")
            print(f"  Refactored result shape: {result_ref.shape}, dtype: {result_ref.dtype}")
    else:
        print("Comparison skipped due to missing results.")

    print("-" * 60)
    return time_orig_avg, time_ref_avg


if __name__ == "__main__":
    # --- Configuration ---
    grid_sizes = [(32, 32), (64, 64), (128, 128), (256, 256)] # Test different sizes
    trials = 100 # Number of repetitions for timing

    # --- Run Comparisons ---
    results = {}
    for size in grid_sizes:
        iicxn, iicyn = size
        try:
            orig_t, ref_t = run_numba_comparison(iicxn, iicyn, num_trials=trials)
            results[size] = {'original_time': orig_t, 'refactored_time': ref_t}
        except Exception as e:
            print(f"\n!!!!!! Error during comparison for size {size}: {e} !!!!!!\n")
            results[size] = {'original_time': float('inf'), 'refactored_time': float('inf')}


    # --- Print Summary ---
    print("\n" + "=" * 60)
    print("Numba Comparison Summary (Average Times in seconds)")
    print("=" * 60)
    print(f"{'Grid Size':<12} | {'Original Time':<18} | {'Refactored Time':<18} | {'Speedup (Ref/Orig)':<18}")
    print("-" * 60)
    for size, times in results.items():
        orig_t = times['original_time']
        ref_t = times['refactored_time']
        if orig_t == 0 or orig_t == float('inf') or ref_t == float('inf'):
            speedup_str = "N/A"
        else:
            speedup = orig_t / ref_t
            speedup_str = f"{speedup:.2f}x"
        print(f"{f'{size[0]}x{size[1]}':<12} | {orig_t:.6f}{'s':<12} | {ref_t:.6f}{'s':<12} | {speedup_str:<18}")
    print("=" * 60)

------------------------------------------------------------
Running comparison for grid size: 32 x 32
Number of trials for timing: 100
------------------------------------------------------------
Running Original Numba version...
Original average execution time: 0.000002 seconds

Running Refactored Numba version...
Refactored average execution time: 0.000001 seconds

Comparing results...
Results are close: True
------------------------------------------------------------
------------------------------------------------------------
Running comparison for grid size: 64 x 64
Number of trials for timing: 100
------------------------------------------------------------
Running Original Numba version...
Original average execution time: 0.000002 seconds

Running Refactored Numba version...
Refactored average execution time: 0.000002 seconds

Comparing results...
Results are close: True
------------------------------------------------------------
----------------------------------------------

In [30]:
import numpy as np
from typing import Tuple

def gradient_2d_standalone(p_nodal: np.ndarray, dx: float, dy: float) -> Tuple[np.ndarray, np.ndarray]:
    """
    Calculates the cell-centered gradient of a 2D nodal scalar field.

    Uses the difference-then-average scheme identified previously:
    Dpx[i,j] = 0.5 * ( (p[i+1,j] - p[i,j])/dx + (p[i+1,j+1] - p[i,j+1])/dx )
    Dpy[i,j] = 0.5 * ( (p[i,j+1] - p[i,j])/dy + (p[i+1,j+1] - p[i+1,j])/dy )

    Parameters
    ----------
    p_nodal : np.ndarray
        The 2D nodal scalar field, shape (nx+1, ny+1).
    dx : float
        Grid spacing in the x-direction.
    dy : float
        Grid spacing in the y-direction.

    Returns
    -------
    Tuple[np.ndarray, np.ndarray]
        The gradient components (Dpx_cell, Dpy_cell), both shape (nx, ny),
        representing the gradient at cell centers.
    """
    if p_nodal.ndim != 2:
        raise ValueError(f"Input p_nodal must be 2D, but got shape {p_nodal.shape}")

    # Calculate Dpx (gradient in x, centered in cells)
    # Difference along x on bottom edges (shape nx, ny+1) then slice -> (nx, ny)
    diff_x_bot = (p_nodal[1:, :-1] - p_nodal[:-1, :-1]) / dx
    # Difference along x on top edges (shape nx, ny+1) then slice -> (nx, ny)
    diff_x_top = (p_nodal[1:, 1:] - p_nodal[:-1, 1:]) / dx
    Dpx_cell = 0.5 * (diff_x_bot + diff_x_top)

    # Calculate Dpy (gradient in y, centered in cells)
    # Difference along y on left edges (shape nx+1, ny) then slice -> (nx, ny)
    diff_y_left = (p_nodal[:-1, 1:] - p_nodal[:-1, :-1]) / dy
     # Difference along y on right edges (shape nx+1, ny) then slice -> (nx, ny)
    diff_y_right = (p_nodal[1:, 1:] - p_nodal[1:, :-1]) / dy
    Dpy_cell = 0.5 * (diff_y_left + diff_y_right)

    return Dpx_cell, Dpy_cell


def derivative_2d_standalone(var_cell: np.ndarray, axis: int, dx: float, dy: float) -> np.ndarray:
    """
    Calculates the nodal derivative of a 2D cell-centered variable.

    Computes differences between adjacent cell centers along 'axis' (giving
    values on intermediate faces), then averages these face values along the
    other axis to get the result at interior nodes.

    Parameters
    ----------
    var_cell : np.ndarray
        The 2D cell-centered variable, shape (nx, ny).
    axis : int
        Axis along which to compute the derivative (0 for x, 1 for y).
    dx : float
        Grid spacing in the x-direction.
    dy : float
        Grid spacing in the y-direction.

    Returns
    -------
    np.ndarray
        The nodal derivative, defined on interior nodes. Shape (nx-1, ny-1).
    """
    if var_cell.ndim != 2:
        raise ValueError(f"Input var_cell must be 2D, but got shape {var_cell.shape}")
    if axis not in [0, 1]:
         raise ValueError(f"Axis must be 0 or 1, but got {axis}")

    # Determine grid spacing for the derivative axis
    ds = dx if axis == 0 else dy

    # Compute the primary difference between cell centers along the specified axis.
    # This gives values located on the faces between the differenced cells.
    # If axis=0, shape is (nx-1, ny). If axis=1, shape is (nx, ny-1).
    diff_on_faces = np.diff(var_cell, axis=axis) / ds

    # Average these face values along the *other* axis to get nodal values.
    if axis == 0:
        # diff_on_faces has shape (nx-1, ny). Average along axis 1 (y-direction).
        # Average vertical faces values horizontally to get value at node.
        result_node = 0.5 * (diff_on_faces[:, :-1] + diff_on_faces[:, 1:])
    else: # axis == 1
        # diff_on_faces has shape (nx, ny-1). Average along axis 0 (x-direction).
        # Average horizontal faces values vertically to get value at node.
        result_node = 0.5 * (diff_on_faces[:-1, :] + diff_on_faces[1:, :])

    # The result is shape (nx-1, ny-1), located at interior nodes.
    return result_node

# --- Example Usage ---
if __name__ == '__main__':
    nx, ny = 5, 4  # Number of cells
    dx, dy = 0.2, 0.25

    # Create a sample nodal field (nx+1, ny+1)
    x_nodes = np.linspace(0, nx * dx, nx + 1)
    y_nodes = np.linspace(0, ny * dy, ny + 1)
    Xn, Yn = np.meshgrid(x_nodes, y_nodes, indexing='ij')
    p_nodal_test = np.sin(np.pi * Xn) * np.cos(np.pi * Yn / 2.0)

    print("--- Testing gradient_2d_standalone ---")
    Dpx, Dpy = gradient_2d_standalone(p_nodal_test, dx, dy)
    print(f"Input p_nodal shape: {p_nodal_test.shape}")
    print(f"Output Dpx shape: {Dpx.shape}") # Should be (nx, ny) = (5, 4)
    print(f"Output Dpy shape: {Dpy.shape}") # Should be (nx, ny) = (5, 4)
    # print("Dpx:\n", Dpx)
    # print("Dpy:\n", Dpy)

    print("\n--- Testing derivative_2d_standalone ---")
    # Use Dpx as a sample cell-centered field
    dDpx_dx = derivative_2d_standalone(Dpx, axis=0, dx=dx, dy=dy)
    dDpx_dy = derivative_2d_standalone(Dpx, axis=1, dx=dx, dy=dy)
    print(f"Input Dpx shape: {Dpx.shape}")
    print(f"Output d(Dpx)/dx shape: {dDpx_dx.shape}") # Should be (nx-1, ny-1) = (4, 3)
    print(f"Output d(Dpx)/dy shape: {dDpx_dy.shape}") # Should be (nx-1, ny-1) = (4, 3)
    # print("d(Dpx)/dx:\n", dDpx_dx)
    # print("d(Dpx)/dy:\n", dDpx_dy)

    # --- Example: Construct a simple Laplacian (Div(Grad(p))) ---
    # This ignores coefficients and boundary complexities for illustration
    Dpx_c, Dpy_c = gradient_2d_standalone(p_nodal_test, dx, dy)
    dDpx_dx_n = derivative_2d_standalone(Dpx_c, axis=0, dx=dx, dy=dy)
    dDpy_dy_n = derivative_2d_standalone(Dpy_c, axis=1, dx=dx, dy=dy)
    laplacian_p_nodes_interior = dDpx_dx_n + dDpy_dy_n
    print("\n--- Simple Laplacian (Interior Nodes) ---")
    print(f"Laplacian(p) shape: {laplacian_p_nodes_interior.shape}") # Should be (nx-1, ny-1) = (4, 3)
    # print("Laplacian(p):\n", laplacian_p_nodes_interior)

--- Testing gradient_2d_standalone ---
Input p_nodal shape: (6, 5)
Output Dpx shape: (5, 4)
Output Dpy shape: (5, 4)

--- Testing derivative_2d_standalone ---
Input Dpx shape: (5, 4)
Output d(Dpx)/dx shape: (4, 3)
Output d(Dpx)/dy shape: (4, 3)

--- Simple Laplacian (Interior Nodes) ---
Laplacian(p) shape: (4, 3)


In [32]:
import numpy as np
# Assume gradient and derivative functions exist and work as described above
# from your_module import gradient, derivative

def laplacian_naive_composed(
    p_node_full: np.ndarray,  # Nodal p including boundaries, shape (nx+1, ny+1)
    nx: int,                  # Number of cells/internal nodes in x
    ny: int,                  # Number of cells/internal nodes in y
    # --- Coefficients (Assume correctly located) ---
    hplusx_cell: np.ndarray,  # Shape (nx, ny)
    hplusy_cell: np.ndarray,  # Shape (nx, ny)
    hcenter_node_int: np.ndarray,# Shape (nx, ny) - On internal nodes
    diag_inv_node_int: np.ndarray, # Shape (nx, ny) - On internal nodes
    coriolis_cell: tuple,     # Tuple (cyy, cxx, cyx, cxy), each shape (nx, ny)
    # --- Grid Info ---
    dxyz: tuple,              # (dx, dy, dz) - only dx, dy used here
    ndim: int = 2
    # --- Boundary Flags (Assumed handled by gradient/derivative internals) ---
    # x_periodic, y_periodic, x_wall, y_wall
) -> np.ndarray:
    """
    Naive implementation of Laplacian operator by composing gradient and derivative.

    Computes Result ≈ diag_inv * [ Div( C * Grad(p) ) + hcenter * p ]

    WARNING: This is for structural understanding. It is NOT performant
             due to intermediate arrays and may differ numerically from
             a fused stencil implementation like lap2D_gather.
             Assumes gradient/derivative handle boundaries correctly.
    """
    if ndim != 2:
        raise NotImplementedError("This naive version is only for 2D")

    dx, dy = dxyz[0], dxyz[1]
    cyy_cell, cxx_cell, cyx_cell, cxy_cell = coriolis_cell

    # --- Step 1: Gradient (Node -> Cell Center) ---
    # Assumes gradient function handles boundaries based on p_node_full size/padding
    Dpx_cell, Dpy_cell, _ = gradient_2d_standalone(p_node_full, ndim, dxyz) # Output shape (nx, ny)

    # --- Step 2: Apply Coefficients to get Cell-Centered Flux ---
    # Fx = cxx*(hpx*Dpx) + cxy*(hpy*Dpy)
    # Fy = cyx*(hpx*Dpx) + cyy*(hpy*Dpy)
    term_x = hplusx_cell * Dpx_cell
    term_y = hplusy_cell * Dpy_cell

    Fx_cell = cxx_cell * term_x + cxy_cell * term_y
    Fy_cell = cyx_cell * term_x + cyy_cell * term_y
    # Fx_cell, Fy_cell have shape (nx, ny)

    # --- Step 3: Divergence (Cell Center -> Node Center) ---
    # Assumes derivative returns result on internal nodes, shape (nx, ny)
    # Assumes derivative function handles boundaries correctly
    div_Fx_node = derivative_2d_standalone(Fx_cell, axis=0, ndim=ndim, dxyz=dxyz)
    div_Fy_node = derivative_2d_standalone(Fy_cell, axis=1, ndim=ndim, dxyz=dxyz)
    # div_Fx_node, div_Fy_node assumed shape (nx, ny)

    divergence_node = div_Fx_node + div_Fy_node # Shape (nx, ny)

    # --- Step 4: Add Center Term & Apply Preconditioner ---
    # Need p on the same internal nodes as the divergence result
    # Extract internal p values from the full nodal array
    # Assuming internal nodes correspond to indices 1:-1 or 0:nx
    # Let's assume indices 0:nx, 0:ny match the DoF count nx*ny
    p_node_int = p_node_full[0:nx, 0:ny] # Example slicing - NEEDS VERIFICATION

    lap_val_node = divergence_node + hcenter_node_int * p_node_int
    result_node_int = lap_val_node * diag_inv_node_int # Shape (nx, ny)

    # --- Step 5: Flatten result ---
    # Return a 1D array consistent with solvers / lap2D_gather output
    return result_node_int.flatten()

In [35]:
def compare_laplacian_results(iicxn, iicyn):
    """
    Compares the results of the fused Numba and naive composed implementations.
    """
    print("-" * 60)
    print(f"Comparing results for grid size: {iicxn} x {iicyn}")
    print("-" * 60)

    nx, ny = iicxn, iicyn # Internal node counts often match cell counts
    ngnc = nx * ny      # Number of internal grid nodes / DoFs

    # --- Generate Consistent Random Test Data (CPU) ---
    np.random.seed(42)
    # Nodal array including boundary nodes (size (nx+1) x (ny+1))
    p_nodal_full = np.random.rand(nx + 1, ny + 1).astype(np.float64)
    # Flattened internal node vector for fused version (size ngnc)
    # Assuming internal nodes are indexed [1:-1, 1:-1] in the full nodal array
    # Or adjust if internal nodes correspond to [0:nx, 0:ny]
    # Let's assume the DoFs correspond to nodes 0..nx-1, 0..ny-1 for simplicity
    # Need to be consistent with how lap2D_gather_fused expects its input
    # If lap2D_gather_fused works on p[0:nx, 0:ny] flattened:
    p_internal_flat = p_nodal_full[0:nx, 0:ny].flatten()

    # Coefficients
    # Cell-centered coeffs for naive version (shape (nx, ny))
    hplusx_cell = np.random.rand(nx, ny).astype(np.float64) + 0.1
    hplusy_cell = np.random.rand(nx, ny).astype(np.float64) + 0.1
    coriolis_cell = tuple([(np.random.rand(nx, ny) - 0.5) * 0.1 for _ in range(4)])

    # Nodal coeffs on internal nodes for naive version (shape (nx, ny))
    hcenter_node_int = np.random.rand(nx, ny).astype(np.float64) * 5.0 + 1.0
    diag_inv_node_int = (1.0 / hcenter_node_int).astype(np.float64)

    # Coeffs for fused version (might be element or node based, need consistency)
    # Assuming fused version needs element-based (like cells), shape (nx, ny)
    hplusx_fused = hplusx_cell
    hplusy_fused = hplusy_cell
    # Assuming fused version needs nodal center/diag_inv flattened (size ngnc)
    hcenter_fused = hcenter_node_int.flatten()
    diag_inv_fused = diag_inv_node_int.flatten()
    # Assuming fused version needs coriolis flattened or passed as tuple of cell arrays
    coriolis_fused = tuple(c.flatten() for c in coriolis_cell) # Example: Flatten if needed
    # *** Crucially, ensure the coefficients passed to both versions represent
    # *** the SAME physical quantities at the SAME logical locations.

    # Grid spacing and boundary conditions
    dx = 1.0 / nx
    dy = 1.0 / ny
    oodx = 1.0 / dx
    oody = 1.0 / dy
    dxyz = (dx, dy, 0.0)
    x_periodic, y_periodic = False, False # Test with walls first maybe
    x_wall, y_wall = True, True

    # --- 1. Run Fused Numba Version ---
    print("Running Fused Numba version (lap2D_gather)...")
    try:
        # Note: lap2D_gather needs iicxn=nx, iicyn=ny for internal node counts
        result_fused = lap2D_gather(
            p_internal_flat, nx, ny,
            hplusx_fused, hplusy_fused, hcenter_fused, oodx, oody,
            x_periodic, y_periodic, x_wall, y_wall, diag_inv_fused, coriolis_fused
        )
        print("Fused version ran.")
    except Exception as e:
        print(f"Error running fused version: {e}")
        result_fused = None


    # --- 2. Run Naive Composed Version ---
    print("\nRunning Naive Composed version (laplacian_naive_composed)...")
    try:
        # Pass full nodal p and cell/internal node coefficients
        result_composed = laplacian_naive_composed(
            p_nodal_full, nx, ny,
            hplusx_cell, hplusy_cell, hcenter_node_int, diag_inv_node_int,
            coriolis_cell, dxyz, ndim=2
        )
        print("Composed version ran.")
    except Exception as e:
        print(f"Error running composed version: {e}")
        result_composed = None


    # --- 3. Compare Results ---
    print("\nComparing results...")
    if result_fused is not None and result_composed is not None:
        if result_fused.shape != result_composed.shape:
            print(" !!! Result shapes differ !!!")
            print(f"  Fused shape:    {result_fused.shape}")
            print(f"  Composed shape: {result_composed.shape}")
            print(" Cannot compare results.")
        else:
            try:
                # Use a relatively loose tolerance initially, as numerical paths differ
                are_close = np.allclose(result_fused, result_composed, rtol=1e-5, atol=1e-7)
                print(f"Results are close (rtol=1e-5, atol=1e-7): {are_close}")
                if not are_close:
                    diff = np.abs(result_fused - result_composed)
                    max_abs_diff = np.max(diff)
                    # Add small epsilon to denominator to avoid division by zero/large rel diff for small numbers
                    denom = np.maximum(np.abs(result_fused), np.abs(result_composed)) + 1e-15
                    max_rel_diff = np.max(diff / denom)

                    print(f"  Max absolute difference: {max_abs_diff:.2e}")
                    print(f"  Max relative difference: {max_rel_diff:.2e}")
            except Exception as e:
                print(f"Error during comparison: {e}")
                print(f"  Fused result dtype:    {result_fused.dtype}")
                print(f"  Composed result dtype: {result_composed.dtype}")
    else:
        print("Comparison skipped due to missing results or errors during execution.")

    print("-" * 60)


if __name__ == "__main__":
    # --- Configuration ---
    # Use a small grid size for initial comparison
    grid_size_compare = (16, 16)

    # --- Run Comparison ---
    compare_laplacian_results(grid_size_compare[0], grid_size_compare[1])

------------------------------------------------------------
Comparing results for grid size: 16 x 16
------------------------------------------------------------
Running Fused Numba version (lap2D_gather)...
Fused version ran.

Running Naive Composed version (laplacian_naive_composed)...
Error running composed version: operands could not be broadcast together with shapes (16,16) (3,) 

Comparing results...
Comparison skipped due to missing results or errors during execution.
------------------------------------------------------------
